In [1]:
import pandas as pd
import glob
import os
import numpy as np

In [2]:
import datetime
import timestring
from dateutil.relativedelta import relativedelta

import plotly.express as pt

import matplotlib.pyplot as plt
plt.rcParams['figure.figsize'] = (12,6)

import seaborn as sns

import warnings
warnings.filterwarnings("ignore")

In [3]:
import random

In [4]:
from faker import Faker

In [5]:
df = pd.read_hdf('df_input_class.h5')

In [6]:
df

,ID,checkout,checkout_at,customer_no,day,enter_at,entry,location,timestamp,week_code,time_spent,sequence,tod,shift,time_shift
0,000010001,False,2019-09-04 07:02:00,1,wednesday,2019-09-04 07:00:00,True,checkin,2019-09-04 07:00:00,00001,0.0,0,morning,fruit,2019-09-04 07:00:00
1,000010001,False,2019-09-04 07:02:00,1,wednesday,2019-09-04 07:00:00,True,fruit,2019-09-04 07:00:00,00001,2.0,1,morning,checkout,2019-09-04 07:02:00
2,000010001,True,2019-09-04 07:02:00,1,wednesday,2019-09-04 07:00:00,False,checkout,2019-09-04 07:02:00,00001,2.0,2,morning,NaN,NaT
3,000010002,False,2019-09-04 07:06:00,2,wednesday,2019-09-04 07:00:00,True,checkin,2019-09-04 07:00:00,00001,0.0,0,morning,dairy,2019-09-04 07:00:00
4,000010002,False,2019-09-04 07:06:00,2,wednesday,2019-09-04 07:00:00,True,dairy,2019-09-04 07:00:00,00001,5.0,1,morning,spices,2019-09-04 07:05:00
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
32319,100001509,True,2019-09-06 21:50:00,1509,friday,2019-09-06 21:50:00,True,drinks,2019-09-06 21:50:00,10000,1.0,1,evening,checkout,2019-09-06 21:51:00
32348,100001509,True,2019-09-06 21:50:00,1509,friday,2019-09-06 21:50:00,True,checkout,2019-09-06 21:51:00,10000,1.0,2,evening,NaN,NaT
32320,100001510,False,2019-09-06 21:50:00,1510,friday,2019-09-06 21:50:00,True,checkin,2019-09-06 21:50:00,10000,0.0,0,evening,spices,2019-09-06 21:50:00
32321,100001510,True,2019-09-06 21:50:00,1510,friday,2019-09-06 21:50:00,True,spices,2019-09-06 21:50:00,10000,1.0,1,evening,checkout,2019-09-06 21:51:00


In [7]:
df.columns

Index(['ID', 'checkout', 'checkout_at', 'customer_no', 'day', 'enter_at',
       'entry', 'location', 'timestamp', 'week_code', 'time_spent', 'sequence',
       'tod', 'shift', 'time_shift'],
      dtype='object')

In [8]:
#df.groupby('day')['time_spent'].d()

In [9]:
p_mat = pd.read_csv('t_matrix.csv', index_col=0)

In [10]:
p_mat

,checkout,dairy,drinks,fruit,spices
location,,,,,
checkin,0.000000,0.287576,0.153526,0.377435,0.181464
dairy,0.393033,0.000000,0.222483,0.189357,0.195127
drinks,0.537260,0.027145,0.000000,0.218950,0.216645
fruit,0.500195,0.237993,0.136080,0.000000,0.125732
spices,0.251998,0.323122,0.272776,0.152104,0.000000


In [11]:
#p_mat = p_mat.T

In [12]:
p_mat.loc['dairy']

checkout    0.393033
dairy       0.000000
drinks      0.222483
fruit       0.189357
spices      0.195127
Name: dairy, dtype: float64

In [13]:
time_spent = pd.read_csv('time_spent.csv', index_col=0)

In [14]:
time_spent.drop('checkin', axis=0, inplace=True)

In [15]:
#time_spent.drop('0.0', axis=1, inplace=True)

In [16]:
time_spent

,0.0,1.0,2.0,3.0,4.0,5.0,6.0,7.0,8.0,9.0,...,19.0,20.0,21.0,23.0,24.0,25.0,26.0,27.0,28.0,30.0
location,,,,,,,,,,,,,,,,,,,,,
checkout,0.0,0.392478,0.231833,0.141169,0.078576,0.049966,0.035057,0.024177,0.014506,0.011283,...,0.000403,0.000134,0.000000,0.000269,0.000134,0.000000,0.000000,0.000000,0.000000,0.000134
dairy,0.0,0.296217,0.185937,0.134217,0.090831,0.070314,0.050652,0.045095,0.030135,0.027143,...,0.001496,0.000427,0.000427,0.000641,0.000427,0.000214,0.000641,0.000214,0.000214,0.000214
drinks,0.0,0.388220,0.255570,0.147759,0.082458,0.050192,0.030218,0.016901,0.011268,0.008195,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
fruit,0.0,0.410972,0.229988,0.147403,0.083756,0.047052,0.034947,0.020109,0.011519,0.005662,...,0.000390,0.000195,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
spices,0.0,0.600959,0.238146,0.096430,0.039158,0.012520,0.007991,0.002930,0.000533,0.000266,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000


In [17]:
time_spent.index

Index(['checkout', 'dairy', 'drinks', 'fruit', 'spices'], dtype='object', name='location')

In [18]:
p_mat.index

Index(['checkin', 'dairy', 'drinks', 'fruit', 'spices'], dtype='object', name='location')

In [24]:
locations = ['dairy', 'drinks', 'fruit', 'spices', 'checkout']

In [253]:
class Customer:
    """
    a single customer that moves through the supermarket
    in a MC simulation
    """
    ...
    
    def __init__(self, name):
        self.name = name
        self.location = 'checkin'
        #self.time = 1.0
        self.gen = self.markov()
    
    def __repr__(self):
        return f'{self.name} is entering in a supermarket through the {self.location} gate and heading towards'

    def get_next_location(self):
        return next(self.gen)

    def markov(self):

        while self.location != 'checkout':

            # calculate the next location
            next_location = np.random.choice(locations, 1, p=p_mat.loc[f'{self.location}'])[0]
            #time_required = np.random.choice(locations, 1, p=time_spent.loc[f'{self.location}'])[0]

            if next_location == 'checkout':
                self.location = 'checkout'
                #self.time = time_required
                yield f'exchanged smiles, paid and thanked to the cashier at {self.location} point'

            else:
                self.location = next_location
                #self.time = time_required
                yield f'{self.location} section'

In [21]:
faker = Faker()

In [300]:
cust1 = Customer(faker.name())
print(cust1)

Priscilla Watson is entering in a supermarket through the checkin gate and heading towards


In [307]:
cust1.get_next_location()

'exchanged smiles, paid and thanked to the cashier at checkout point'